In [8]:
import wandb
wandb.login()

wandb: Currently logged in as: johnwilsonlorin1 (johnwilsonlorin1-laguna-state-polytechnic-university). Use `wandb login --relogin` to force relogin


True

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import pickle 
import wandb


wandb.init(project='LLDA-Detecting Algal Blooms and Fish Kill', name = "SVR")

# Load datasets with specified encoding
merged_df = pd.read_csv('complete.csv')

# Select relevant features and target
features = ['Temperature', 'Humidity', 'Wind Speed', 'pH (units)', 'Ammonia (mg/L)', 'Inorganic Phosphate (mg/L)', 'BOD (mg/l)', 'Total coliforms (MPN/100ml)']
target = 'Phytoplankton (cells/ml)'

merged_df = merged_df.dropna()

# Split data into training and testing sets
X = merged_df[features]
y = merged_df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define parameter grid for Grid Search with SVR
param_grid = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [1, 10, 100],
    'epsilon': [0.1, 0.2, 0.3],
    'gamma': ['scale', 'auto']
}

# Initialize GridSearchCV with SVR
grid_search = GridSearchCV(SVR(), param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit GridSearchCV
grid_search.fit(X_train_scaled, y_train)

# Best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Score (Negative MSE):", grid_search.best_score_)

# Train model with best parameters
best_svr_model = grid_search.best_estimator_

# Make predictions and evaluate
y_pred_best_svr = best_svr_model.predict(X_test_scaled)
mse_best_svr = mean_squared_error(y_test, y_pred_best_svr)
mae_best_svr = mean_absolute_error(y_test, y_pred_best_svr)
r2_best_svr = r2_score(y_test, y_pred_best_svr)

print(f'Optimized SVR - Mean Squared Error: {mse_best_svr}')
print(f'Optimized SVR - Mean Absolute Error: {mae_best_svr}')
print(f'Optimized SVR - R^2 Score: {r2_best_svr}')

# Log hyperparameters and metrics to wandb
wandb.log({
    'MSE': mse_best_svr,
    'MAE': mae_best_svr,
    'R2 Score': r2_best_svr
})

# Finish the wandb run
wandb.finish()



Best Parameters: {'C': 100, 'epsilon': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
Best Score (Negative MSE): -12503041110.180258
Optimized SVR - Mean Squared Error: 7998157399.400708
Optimized SVR - Mean Absolute Error: 30207.92797770837
Optimized SVR - R^2 Score: -0.030744768486453244


MAE,▁
MSE,▁
R2 Score,▁
MAE,30207.92798
MSE,7998157399.40071
R2 Score,-0.03074


In [5]:
import pickle

In [13]:
with open('best_svr_model.pkl', 'wb') as file:
    pickle.dump(best_svr_model, file)

In [15]:
with open('best_svr_model.pkl', 'rb') as file:
    model_loaded = pickle.load(file)

In [37]:
model_loaded.predict(X_test_scaled)

array([26877.91341235,  9771.28544985, 26533.35374031,  5037.03887016,
       22414.24656182, 35397.90630199, 16478.50761971, 13293.06401851,
        7989.5997163 , 37192.28850061, 27230.47315956, 11594.6917227 ,
       18888.42394755, 27421.0647427 , 30474.64200167,  3898.7479411 ,
        2820.75191692, 19536.56823213, 14179.3159015 ,  5319.40243479,
       12915.34891703, 21348.70281201, 12369.34916928, 18551.4853526 ,
       14851.1650974 ,  6966.0383026 , 12790.44648884, 23621.19357102,
       14144.7395092 , 18091.55150746, 29078.1329786 , 24309.94674064,
       13155.48560273, 12086.54546426, 13439.52615793, 19264.30387838,
       16087.44043816, 21852.4603923 , 11236.2531948 , 24869.26763311,
       13952.80131416, 18406.50422039, 21539.51004686,  7601.68940917,
        7281.17619539, 15073.52260182, 16538.60283171, 24137.32043052,
       14149.61261053, 19900.07895384, 17666.11756913, 17569.89826596,
       20648.94059326, 18772.03287399,  2171.09066793, 22218.10946547,
      

In [43]:
# Save the scaler
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)
